In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.preprocessing    import OneHotEncoder
from sklearn.preprocessing    import StandardScaler
from sklearn.preprocessing    import Normalizer
from sklearn.compose          import ColumnTransformer

In [3]:
from sklearn.pipeline         import Pipeline
from sklearn.model_selection  import GridSearchCV
from sklearn.model_selection  import KFold
from sklearn.model_selection  import cross_val_score

In [4]:
from sklearn.metrics          import make_scorer
from sklearn.metrics          import mean_squared_error
from sklearn.metrics          import mean_squared_log_error
from sklearn.metrics          import r2_score

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import matplotlib.pyplot as     plt

In [7]:
from sklearn.ensemble       import GradientBoostingRegressor
from sklearn.svm            import SVR
from sklearn.linear_model   import Lasso
from sklearn.neighbors      import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

In [8]:
from math import sqrt, log, exp

In [9]:
#read data
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

In [10]:
# store the copy
train_copy = train.copy()
test_copy  = test .copy()

In [11]:
test.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [12]:
train.shape,test.shape

((1460, 81), (1459, 80))

In [13]:
# get index, set ID as index
id_train = train['Id']
id_test  = test ['Id']

train.set_index('Id', drop = True, inplace = True)
test .set_index('Id', drop = True, inplace = True)

print("train:",train.index,train.columns)
print("test:" ,test .index,test .columns)


train: Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460],
           dtype='int64', name='Id', length=1460) Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbv

In [14]:
# separete y, check for null 
y = train['SalePrice']
train.drop('SalePrice', axis =1, inplace = True)
id_y = y.index   
y.isnull().sum()

0

Look at date

In [15]:
# Look at data
train.shape,train.columns,train.dtypes,train.head(5)

((1460, 79),
 Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
        'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
        'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
        'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
        'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
        'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
        'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
        'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
        'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
        'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
        'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
        'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
        'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond'

Separate var by types

In [16]:
# columns' list of types object, int, float
columns_obj    = [column for column in train.columns if train[column].dtypes == 'object']
columns_int    = [column for column in train.columns if train[column].dtypes == 'int64' ]
columns_float  = [column for column in train.columns if train[column].dtypes == 'float' ]
len(train.columns.to_list()),len(columns_obj),len(columns_int),len(columns_float)

(79, 43, 33, 3)

In [17]:
# look at columns_int to separate categorial, range, numerical variables
train[columns_int].shape,train[columns_int],columns_int

((1460, 33),
       MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  \
 Id                                                                             
 1             60     8450            7            5       2003          2003   
 2             20     9600            6            8       1976          1976   
 3             60    11250            7            5       2001          2002   
 4             70     9550            7            5       1915          1970   
 5             60    14260            8            5       2000          2000   
 ...          ...      ...          ...          ...        ...           ...   
 1456          60     7917            6            5       1999          2000   
 1457          20    13175            6            6       1978          1988   
 1458          70     9042            7            9       1941          2006   
 1459          20     9717            5            6       1950          1996   
 1460          

In [18]:
for column in columns_int: print(column,train[column].nunique())

MSSubClass 15
LotArea 1073
OverallQual 10
OverallCond 9
YearBuilt 112
YearRemodAdd 61
BsmtFinSF1 637
BsmtFinSF2 144
BsmtUnfSF 780
TotalBsmtSF 721
1stFlrSF 753
2ndFlrSF 417
LowQualFinSF 24
GrLivArea 861
BsmtFullBath 4
BsmtHalfBath 3
FullBath 4
HalfBath 3
BedroomAbvGr 8
KitchenAbvGr 4
TotRmsAbvGrd 12
Fireplaces 4
GarageCars 5
GarageArea 441
WoodDeckSF 274
OpenPorchSF 202
EnclosedPorch 120
3SsnPorch 20
ScreenPorch 76
PoolArea 8
MiscVal 21
MoSold 12
YrSold 5


In [19]:
# select different types of  int variables: cat, range, numerical.
columns_int_years            = ['YearBuilt','YearRemodAdd','YrSold']
columns_not_result_not_year  = [column for column in columns_int                 if column not in columns_int_years] 
columns_int_numerical        = [column for column in columns_not_result_not_year if train[column].nunique()>15]
columns_int_not_num          = [column for column in columns_not_result_not_year if column not in columns_int_numerical]   
for column in columns_int_not_num: print(column,train[column].nunique())
len(columns_int_not_num)

MSSubClass 15
OverallQual 10
OverallCond 9
BsmtFullBath 4
BsmtHalfBath 3
FullBath 4
HalfBath 3
BedroomAbvGr 8
KitchenAbvGr 4
TotRmsAbvGrd 12
Fireplaces 4
GarageCars 5
PoolArea 8
MoSold 12


14

In [20]:
len(columns_int),len(columns_int_years),len(columns_not_result_not_year),len(columns_int_numerical),len(columns_int_not_num)

(33, 3, 30, 16, 14)

In [21]:
columns_int_numerical

['LotArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'MiscVal']

In [22]:
train['BsmtFullBath'].unique(),train['BsmtHalfBath'].unique()
train[['BsmtFullBath','FullBath']].corr(method ='spearman')

,BsmtFullBath,FullBath
BsmtFullBath,1.000000,-0.056379
FullBath,-0.056379,1.000000


In [23]:
columns_int_numerical.append('PoolArea'  )
columns_int_numerical.append('GarageCars')
columns_int_numerical.append('Fireplaces')
#columns_int_numerical.append('BsmtFullBath')
#columns_int_numerical.append('BsmtHalfBath')
#columns_int_numerical.append('FullBath')
#columns_int_numerical.append('HalfBath')
columns_int_numerical.append('TotRmsAbvGrd')
columns_int_numerical.append('BedroomAbvGr')
columns_int_numerical.append('KitchenAbvGr')

In [24]:
columns_rest_3 = [column for column in columns_not_result_not_year if column not in columns_int_numerical]   
for column in columns_rest_3: print(column,train[column].nunique())
len(columns_rest_3)

MSSubClass 15
OverallQual 10
OverallCond 9
BsmtFullBath 4
BsmtHalfBath 3
FullBath 4
HalfBath 3
MoSold 12


8

In [25]:
len(columns_int),len(columns_int_years),len(columns_not_result_not_year),len(columns_int_numerical),len(columns_rest_3)

(33, 3, 30, 22, 8)

In [26]:
columns_num_range = ['MoSold','OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath']
# try put "Baths" into cat!!!
columns_num_cat   = ['MSSubClass',]

In [27]:
# check if all columns are classified
rest = [column for column in train.columns if (
        (column not in
         columns_obj+
         columns_float+
         columns_int_numerical+
         columns_num_range+
         columns_num_cat+
         columns_int_years)
       )]
rest

[]

In [28]:
columns_dif_types_x = {'columns_obj':columns_obj,
                     'columns_float':columns_float,
                     'columns_int_numerical':columns_int_numerical,
                     'columns_num_range':columns_num_range,
                     'columns_num_cat':columns_num_cat,
                     'columns_int_years':columns_int_years}

Check for null

In [29]:
# To check for null we put together all three data set
dataset = pd.concat([train,test],axis =0)
train.shape,test.shape,dataset.shape


((1460, 79), (1459, 79), (2919, 79))

In [30]:
dataset.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [31]:
ind_dataset_list = list(dataset.index)

In [32]:
train.index,test.index

(Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
             ...
             1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460],
            dtype='int64', name='Id', length=1460),
 Int64Index([1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470,
             ...
             2910, 2911, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919],
            dtype='int64', name='Id', length=1459))

In [33]:
# Check for null
column_list = []
null_n_list = []
null_percents_list = []
dtype_list =[]

N = dataset.shape[0]
M = dataset.shape[1]
print(N,M)
for key in columns_dif_types_x.keys():
    col_list = columns_dif_types_x[key]
    #print(f'{key}\n{col_list}\n')
    for column in col_list:
        null_n      = dataset[column].isnull().sum()
        
        # null can absent in train set , but be in test set
        # in this case voluntary assing '1' to null_n, to work out these columns
     
        null_percents = null_n/N
        column_list.append(column)
        null_n_list.append(null_n)
        null_percents_list.append(null_percents)
        dtype_list.append(key) 
        
null_stat = pd.DataFrame({'dtype':dtype_list,
                         'column':column_list,
                         'null_n':null_n_list,
                         'null_percents':null_percents_list})
null_stat_sorted = null_stat.sort_values('null_percents',ascending = False)
print(null_stat_sorted.shape)
null_stat_sorted.head(50)

2919 79
(79, 4)


,dtype,column,null_n,null_percents
38,columns_obj,PoolQC,2909,0.996574
40,columns_obj,MiscFeature,2814,0.964029
2,columns_obj,Alley,2721,0.932169
39,columns_obj,Fence,2348,0.804385
32,columns_obj,FireplaceQu,1420,0.486468
43,columns_float,LotFrontage,486,0.166495
36,columns_obj,GarageCond,159,0.054471
34,columns_obj,GarageFinish,159,0.054471
35,columns_obj,GarageQual,159,0.054471
45,columns_float,GarageYrBlt,159,0.054471


In [34]:
nulls = null_stat_sorted.loc[null_stat_sorted['null_n']>0]
nulls

,dtype,column,null_n,null_percents
38,columns_obj,PoolQC,2909,0.996574
40,columns_obj,MiscFeature,2814,0.964029
2,columns_obj,Alley,2721,0.932169
39,columns_obj,Fence,2348,0.804385
32,columns_obj,FireplaceQu,1420,0.486468
43,columns_float,LotFrontage,486,0.166495
36,columns_obj,GarageCond,159,0.054471
34,columns_obj,GarageFinish,159,0.054471
35,columns_obj,GarageQual,159,0.054471
45,columns_float,GarageYrBlt,159,0.054471


In [35]:
# Replace null for cat (object - type)
# 1. Many   null
# 2. a few  null

In [36]:
# set threshold many, few
threshold_null = 0.06
# form list of variables
nulls_obj      = nulls[nulls['dtype'] == 'columns_obj']
nulls_obj_many = nulls_obj[nulls_obj['null_percents'] >= threshold_null ]
nulls_obj_few  = nulls_obj[nulls_obj['null_percents'] <  threshold_null ]

In [37]:
nulls_obj_many,nulls_obj_few

(          dtype       column  null_n  null_percents
 38  columns_obj       PoolQC    2909       0.996574
 40  columns_obj  MiscFeature    2814       0.964029
 2   columns_obj        Alley    2721       0.932169
 39  columns_obj        Fence    2348       0.804385
 32  columns_obj  FireplaceQu    1420       0.486468,
           dtype        column  null_n  null_percents
 36  columns_obj    GarageCond     159       0.054471
 34  columns_obj  GarageFinish     159       0.054471
 35  columns_obj    GarageQual     159       0.054471
 33  columns_obj    GarageType     157       0.053786
 23  columns_obj  BsmtExposure      82       0.028092
 22  columns_obj      BsmtCond      82       0.028092
 21  columns_obj      BsmtQual      81       0.027749
 25  columns_obj  BsmtFinType2      80       0.027407
 24  columns_obj  BsmtFinType1      79       0.027064
 17  columns_obj    MasVnrType      24       0.008222
 0   columns_obj      MSZoning       4       0.001370
 5   columns_obj     Utilities   

In [38]:
# replace object's type variable

# few: replace for most common
for column in nulls_obj_few['column']:
    most_frequent = train[column].mode()[0]
    print(column,most_frequent)
    dataset[column].fillna(most_frequent, inplace = True)
        
# many: replace for new label "no_data"
for column in nulls_obj_many['column']:
    dataset[column].fillna('no_data', inplace = True)

dataset[nulls_obj['column']].isnull().sum()

GarageCond TA
GarageFinish Unf
GarageQual TA
GarageType Attchd
BsmtExposure No
BsmtCond TA
BsmtQual TA
BsmtFinType2 Unf
BsmtFinType1 Unf
MasVnrType None
MSZoning RL
Utilities AllPub
Functional Typ
KitchenQual TA
SaleType WD
Electrical SBrkr
Exterior2nd VinylSd
Exterior1st VinylSd


PoolQC          0
MiscFeature     0
Alley           0
Fence           0
FireplaceQu     0
GarageCond      0
GarageFinish    0
GarageQual      0
GarageType      0
BsmtExposure    0
BsmtCond        0
BsmtQual        0
BsmtFinType2    0
BsmtFinType1    0
MasVnrType      0
MSZoning        0
Utilities       0
Functional      0
KitchenQual     0
SaleType        0
Electrical      0
Exterior2nd     0
Exterior1st     0
dtype: int64

In [39]:
columns_dif_types_x.keys()

dict_keys(['columns_obj', 'columns_float', 'columns_int_numerical', 'columns_num_range', 'columns_num_cat', 'columns_int_years'])

In [40]:
print(columns_dif_types_x['columns_num_cat'  ] ,nulls[nulls['dtype'] == 'columns_num_cat']  )
print(columns_dif_types_x['columns_num_range'] ,nulls[nulls['dtype'] == 'columns_num_range'])
print(columns_dif_types_x['columns_int_years'] ,nulls[nulls['dtype'] == 'columns_int_years'])

['MSSubClass'] Empty DataFrame
Columns: [dtype, column, null_n, null_percents]
Index: []
['MoSold', 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath']                 dtype        column  null_n  null_percents
71  columns_num_range  BsmtFullBath       2       0.000685
72  columns_num_range  BsmtHalfBath       2       0.000685
['YearBuilt', 'YearRemodAdd', 'YrSold'] Empty DataFrame
Columns: [dtype, column, null_n, null_percents]
Index: []


In [41]:
# NaN replace for range var
col_num_range_nan = [col for col in columns_num_range if col in list(nulls['column'])]
print(col_num_range_nan)
for column in col_num_range_nan:
    most_frequent = train[column].mode()[0]
    print(column,most_frequent)
    dataset[column].fillna(most_frequent, inplace = True)
#check
dataset[col_num_range_nan].isnull().sum()

['BsmtFullBath', 'BsmtHalfBath']
BsmtFullBath 0
BsmtHalfBath 0


BsmtFullBath    0
BsmtHalfBath    0
dtype: int64

In [42]:
# NaN replace float's type variable
columns_float_plus_num = columns_float + columns_int_numerical
print(columns_float_plus_num)

nulls_float_int  = nulls[(nulls['dtype'] == 'columns_float')|(nulls['dtype'] == 'columns_int_numerical')]
print(f'nulls_float_int:\n{nulls_float_int}\n')

# select columns with null more then theshold
nulls_float_num_many = nulls_float_int[nulls_float_int['null_percents'] >= threshold_null ]
print(f'\nnulls_float_num_many\n{type(nulls_float_num_many)}\n{nulls_float_num_many}')

# new column:  
# test[new_column]  ==1 => data present,
# test[new_column]  ==0 => data absent and were imputed  
for item in nulls_float_num_many['column']:
    print(f"test\ntype(item):{type(item)}\nitem.index:{item.index}\nitem:{item}")
    column = item
    new_column = str(column+'_replace')
    dataset[new_column] = np.where(dataset[column].isnull(),0,1)    

# replace null for mean
for column in nulls_float_int['column']:
    var_mean = train[column].mean()
    print(column,var_mean)
    dataset[column].fillna(var_mean, inplace = True)


['LotFrontage', 'MasVnrArea', 'GarageYrBlt', 'LotArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'MiscVal', 'PoolArea', 'GarageCars', 'Fireplaces', 'TotRmsAbvGrd', 'BedroomAbvGr', 'KitchenAbvGr']
nulls_float_int:
                    dtype       column  null_n  null_percents
43          columns_float  LotFrontage     486       0.166495
45          columns_float  GarageYrBlt     159       0.054471
44          columns_float   MasVnrArea      23       0.007879
49  columns_int_numerical    BsmtUnfSF       1       0.000343
50  columns_int_numerical  TotalBsmtSF       1       0.000343
48  columns_int_numerical   BsmtFinSF2       1       0.000343
47  columns_int_numerical   BsmtFinSF1       1       0.000343
63  columns_int_numerical   GarageCars       1       0.000343
55  columns_int_numerical   GarageArea       1       0.000343


nulls_flo

In [43]:
# check if nun is missing
dataset[dataset.isnull().any(axis=1)],dataset.isnull().sum().sum()
# no nun

(Empty DataFrame
 Columns: [MSSubClass, MSZoning, LotFrontage, LotArea, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, YearBuilt, YearRemodAdd, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, Heating, HeatingQC, CentralAir, Electrical, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, KitchenQual, TotRmsAbvGrd, Functional, Fireplaces, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PavedDrive, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, PoolQC, Fence, MiscFeature, MiscVal, MoSold, YrSold, SaleType, SaleCondition, LotFrontage_replace]
 Index: []
 
 [0 rows x 80 col

check for duplicated lines

In [44]:
# check for duplicated lines
print(dataset.duplicated().sum(),dataset[dataset.duplicated(keep = False) ==True])
ind_range_null = dataset[dataset.duplicated(keep = False) ==True].index
print(ind_range_null)
id_train = [i for i in id_train if i not in ind_range_null]
id_y     = [i for i in id_y     if i not in ind_range_null]
#id_test  = [i for i in id_test  if i not in ind_range_null ]
print(len(id_train),len(id_test),len(list(dataset.index)))
dataset.drop_duplicates(keep = 'last',inplace = True)
y = y[id_y]
# to keep test size, to submit prediction
dataset.duplicated().sum()

2       MSSubClass MSZoning  LotFrontage  LotArea Street    Alley LotShape  \
Id                                                                         
194          160       RM         24.0     2522   Pave  no_data      Reg   
830          160       FV         24.0     2544   Pave     Pave      Reg   
2714         160       FV         24.0     2544   Pave     Pave      Reg   
2866         160       RM         24.0     2522   Pave  no_data      Reg   

     LandContour Utilities LotConfig  ... PoolArea   PoolQC    Fence  \
Id                                    ...                              
194          Lvl    AllPub    Inside  ...        0  no_data  no_data   
830          Lvl    AllPub    Inside  ...        0  no_data  no_data   
2714         Lvl    AllPub    Inside  ...        0  no_data  no_data   
2866         Lvl    AllPub    Inside  ...        0  no_data  no_data   

     MiscFeature MiscVal MoSold  YrSold  SaleType  SaleCondition  \
Id                                      

0

In [45]:
for el in columns_num_cat:
    print(dataset[el].value_counts())

20     1079
60      575
50      287
120     182
30      139
70      128
160     126
80      118
90      109
190      61
85       48
75       23
45       18
180      17
40        6
150       1
Name: MSSubClass, dtype: int64


Check for rare and quasi-constant var

In [46]:
dataset.index 

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            2910, 2911, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919],
           dtype='int64', name='Id', length=2917)

In [47]:
# return to train/test
train         = dataset.loc[id_train]
test          = dataset.loc[id_test ]

In [48]:
#train.columns

In [49]:
def in_not_in(set_in,set_not_in,columns_for_check):
    # determine wich labels is not in all sets 
    dic = {}
    for col in columns_for_check:
        uniq_in          = list(set_in[col]    .unique())
        uniq_not_in_1    = list(set_not_in[col].unique())
       
        in_not_in_list = [el for el in uniq_in if el not in uniq_not_in_1]
        if len(in_not_in_list)>0: dic[col] =in_not_in_list
    return dic        

In [50]:
# Categirial variable obj
# determine wich labels is not in all sets
# form list od rare var to delete
threshold_max_one_lebel_share = 0.95
threshold_rare_lebel_share    = 0.05
N = train.shape[0]
list_cat_columns_to_delete=[]

for item in columns_obj:
    value_s_train       = train[item]  .value_counts()
    index_train         = value_s_train.index.to_list()
    value_count_train   = value_s_train.to_list()
    value_percent_train = [el/N for el in value_count_train]
    # to avoid passtrough rare unique label only in test
    value_rare_train    = value_s_train.index.to_list()
    value_count_df = pd.DataFrame({'value_count'  :value_s_train,
                                   'value_percent':value_percent_train},
                                   index = value_rare_train)
    # check if in fact it is only one lebel
    max_share = value_count_df['value_percent'].max()
    if max_share > threshold_max_one_lebel_share: 
        list_cat_columns_to_delete.append(item)
    else:
    # replace rare lebels for new lebel -'rare'
        for lebel in value_rare_train:
            if value_count_df.loc[lebel,'value_percent'] <= threshold_rare_lebel_share:
                train.loc[train[item] == lebel,item] = 'rare'
                test .loc[test [item] == lebel,item] = 'rare'
                                
# to avoid passtrough rare unique label only in test and vise versa
only_in_train          = in_not_in(train,test ,columns_obj)
only_in_test           = in_not_in(test ,train,columns_obj)

for item in only_in_train.keys():
    train_labels         = only_in_train[item]
    for el in train_labels: train.loc[train[item] == el,item] = 'rare'
for item in only_in_test.keys():    
    test_labels          = only_in_test [item]
    for el in test_labels:  test.loc[test [item] == el,item] = 'rare'

    
print('list_cat_columns_to_delete',list_cat_columns_to_delete)

list_cat_columns_to_delete ['Street', 'Utilities', 'Condition2', 'RoofMatl', 'Heating', 'GarageQual', 'GarageCond', 'PoolQC', 'MiscFeature']


In [51]:
columns_num_cat

['MSSubClass']

In [52]:
for el in columns_num_cat:
    print(train[el].value_counts())

20     536
60     299
50     144
120     87
30      69
160     61
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64


In [53]:
# Categirial rare variable (num) replace for "0"
threshold_max_one_lebel_share     = 0.95
threshold_num_rare_lebel_share    = 0.03
N = train.shape[0]
list_cat_int_columns_to_delete=[]

for item in columns_num_cat:
    
    value_s_train       = train[item].value_counts()
    index_train         = value_s_train.index.to_list()
    value_count_train   = value_s_train.to_list()
    value_percent_train = [el/N for el in value_count_train]
    # to avoid passtrough rare unique label only in test
    value_rare_train    = value_s_train.index.to_list()
    value_count_df = pd.DataFrame({'value_count'  :value_s_train,
                                   'value_percent':value_percent_train},index = value_rare_train)
    # check if in fact it is only one lebel
    max_share = value_count_df['value_percent'].max()
    if max_share > threshold_max_one_lebel_share: 
        list_cat_int_columns_to_delete.append(item)
    else:
    # replace rare lebels for new lebel -'0
        for lebel in value_rare_train:
            if value_count_df.loc[lebel,'value_percent'] <= threshold_rare_lebel_share:
                train.loc[train[item] == lebel,item] = 0
                test .loc[test [item] == lebel,item] = 0
                
# to avoid passtrough rare unique label only in test and vise versa
only_in_train          = in_not_in(train,test ,columns_num_cat)
only_in_test           = in_not_in(test ,train,columns_num_cat)

for item in only_in_train.keys():
    train_labels         = only_in_train[item]
    for el in train_labels: train.loc[train[item] == el,item] = 0
for item in only_in_test.keys():    
    test_labels          = only_in_test [item]
    for el in test_labels:  test.loc[test [item] == el,item] = 0

train[columns_num_cat].astype('object')
test [columns_num_cat].astype('object')
print(list_cat_int_columns_to_delete,list_cat_columns_to_delete)

[] ['Street', 'Utilities', 'Condition2', 'RoofMatl', 'Heating', 'GarageQual', 'GarageCond', 'PoolQC', 'MiscFeature']


In [54]:
# put train, test together back to dataset
dataset = pd.concat([train,test],axis =0)
train.shape,test.shape,dataset.shape

((1458, 80), (1459, 80), (2917, 80))

In [55]:
train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [56]:
# check replacement in var 'OverallCond'
dataset[columns_num_range].dtypes
dataset.loc[dataset['OverallCond'].astype('int64') == 2,'OverallCond'] 
dataset['OverallCond'].astype('int64') == 2

Id
1       False
2       False
3       False
4       False
5       False
        ...  
2915    False
2916    False
2917    False
2918    False
2919    False
Name: OverallCond, Length: 2917, dtype: bool

In [57]:
# rare lebels in range - var
threshold_rare_range_var_min = 4
threshold_rare_range_var_max = 8
print(columns_num_range)
for el in columns_num_range:
    print(dataset[el].value_counts())
for col in columns_num_range:
    if dataset[col].nunique()>7:
        print(col,dataset[col].nunique())
        dataset.loc[dataset[col] <= threshold_rare_range_var_min,col] = threshold_rare_range_var_min     
        dataset.loc[dataset[col] >= threshold_rare_range_var_max,col] = threshold_rare_range_var_max


['MoSold', 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath']
6     503
7     445
5     393
4     279
8     233
3     232
10    173
9     158
11    142
2     133
1     122
12    104
Name: MoSold, dtype: int64
5     825
6     731
7     598
8     342
4     226
9     107
3      40
10     31
2      13
1       4
Name: OverallQual, dtype: int64
5    1643
6     531
7     390
8     144
4     101
3      50
9      41
2      10
1       7
Name: OverallCond, dtype: int64
0.0    1705
1.0    1172
2.0      38
3.0       2
Name: BsmtFullBath, dtype: int64
0.0    2742
1.0     171
2.0       4
Name: BsmtHalfBath, dtype: int64
2    1528
1    1309
3      64
0      12
4       4
Name: FullBath, dtype: int64
0    1833
1    1059
2      25
Name: HalfBath, dtype: int64
MoSold 12
OverallQual 10
OverallCond 9


In [58]:
# put rare labels together
dataset.loc[dataset['BsmtFullBath']==2,'BsmtFullBath'] = 1
dataset.loc[dataset['BsmtFullBath']==3,'BsmtFullBath'] = 1

dataset.loc[dataset['BsmtHalfBath']==2,'BsmtHalfBath'] = 1

dataset.loc[dataset['FullBath']==0,'FullBath'] = 1
dataset.loc[dataset['FullBath']==3,'FullBath'] = 2
dataset.loc[dataset['FullBath']==4,'FullBath'] = 2

dataset.loc[dataset['HalfBath']==2,'HalfBath'] = 1

In [59]:
for el in ['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath']:
    print(dataset[el].value_counts())

0.0    1705
1.0    1212
Name: BsmtFullBath, dtype: int64
0.0    2742
1.0     175
Name: BsmtHalfBath, dtype: int64
2    1596
1    1321
Name: FullBath, dtype: int64
0    1833
1    1084
Name: HalfBath, dtype: int64


In [60]:
# put ['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath'] from range to cat
correct_type = ['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath']
columns_num_range = [col for col in columns_num_range if col not in correct_type]
columns_num_cat = columns_num_cat +correct_type

In [61]:
# quazi constant num int and float var
threshold_max_one_lebel_share = 0.95


N = train.shape[0]
list_num_columns_to_delete=[]
columns_float_int_num = columns_float + columns_int_numerical
for item in columns_float_int_num:
    value_s        = train[item].value_counts()
    value_count    = value_s.to_list()
    value_percent  = [item/N for item in value_count]
    value          = value_s.index.to_list()
    value_count_df = pd.DataFrame({'value_count'  :value_count,
                                   'value_percent':value_percent}, index = value)
    #print(f'\n{item}\n{value_count_df}')
    
    # check if in fact it is only one lebel
    max_share = value_count_df['value_percent'].max()
    if max_share > threshold_max_one_lebel_share: 
        list_num_columns_to_delete.append(item)
list_num_columns_to_delete

['LowQualFinSF', '3SsnPorch', 'MiscVal', 'PoolArea', 'KitchenAbvGr']

In [62]:
# list_to_delete after above preparation
list_to_delete1 = list_num_columns_to_delete + list_cat_columns_to_delete + list_cat_int_columns_to_delete
list_to_delete1

['LowQualFinSF',
 '3SsnPorch',
 'MiscVal',
 'PoolArea',
 'KitchenAbvGr',
 'Street',
 'Utilities',
 'Condition2',
 'RoofMatl',
 'Heating',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'MiscFeature']

In [63]:
# transfomer to encoder mean
from sklearn.base import BaseEstimator, TransformerMixin
class MeanEncoder(BaseEstimator, TransformerMixin):
    def __init__ (self,columns_list =[]):
        self.columns_list = columns_list
        #columns_list
        #print("__init__: {columns_list}".format(columns_list))
        #print("__init__: {self.columns_list}".format(self.columns_list))
        #self.dicts ={}
                       
    def fit(self,X,y):
        print("in ME fit")
        #self.columns_list = columns_list
        #print("in ME fit: {columns_list}".format(columns_list))
        #print("in ME fit: {}".format(self.columns_list))
        
        dicts = {}
        y_name = y.name
        X_y = pd.concat([X,y], axis =1)
        for column in self.columns_list:
            dict_cat = X_y.groupby(column)[y_name].mean().to_dict()
            dicts[column] = dict_cat
        self.dicts = dicts
        #print("in ME fit: self.dicts:\n{}\n".format(self.dicts))
        return self
    
    #def transform(self,df,y):
    def transform(self,X):
        print("in ME transform")
        #print("self.columns_list",self.columns_list)
        #print("X.columns:\n{}\n".format(X.columns))
        #print("X:\n{}\n".format(X))
        #print("self.dicts:\n{}\n".format(self.dicts))
        for column in self.columns_list:
            dict_cat = self.dicts[column]
            #print("dict_cat",dict_cat)
            X[column] = X[column].map(dict_cat)
        #print("X after transform :\n{}\n".format(X))
        return X

In [64]:
# drop unusfull var cat + num

list_to_delete = list_num_columns_to_delete + list_cat_columns_to_delete + list_cat_int_columns_to_delete
dataset.drop(list_to_delete, axis =1 , inplace =True)
list_to_delete,list_to_delete1

(['LowQualFinSF',
  '3SsnPorch',
  'MiscVal',
  'PoolArea',
  'KitchenAbvGr',
  'Street',
  'Utilities',
  'Condition2',
  'RoofMatl',
  'Heating',
  'GarageQual',
  'GarageCond',
  'PoolQC',
  'MiscFeature'],
 ['LowQualFinSF',
  '3SsnPorch',
  'MiscVal',
  'PoolArea',
  'KitchenAbvGr',
  'Street',
  'Utilities',
  'Condition2',
  'RoofMatl',
  'Heating',
  'GarageQual',
  'GarageCond',
  'PoolQC',
  'MiscFeature'])

In [65]:
# correct columns_lists
for key in columns_dif_types_x.keys():
    #if( key == 'columns_obj'): print(f'{key}\n{columns_dif_types_x[key]}\n')
    columns_dif_types_x[key] = [el for el in columns_dif_types_x[key] if el not in list_to_delete]
    #if( key == 'columns_obj'): print(f'{key}\n{columns_dif_types_x[key]}\n')

In [66]:
columns_dif_types_x.keys()
columns_int_numerical

['LotArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'MiscVal',
 'PoolArea',
 'GarageCars',
 'Fireplaces',
 'TotRmsAbvGrd',
 'BedroomAbvGr',
 'KitchenAbvGr']

In [67]:
# let us see correlation
train = dataset.loc[id_train]
train_plus_y = train.copy()
train_plus_y['SalePrice'] = y
#print(train_plus_y.columns)
corr = train_plus_y.corr(method = 'spearman')
print(f"SalePrice corr:\n{corr.loc[abs(corr['SalePrice'])>0.7,'SalePrice']}\n")
#print(corr.index)
list_temp=[]
for i in corr.index:
    for j in corr.columns:
        if (i!='SalePrice') and (j!='SalePrice'):
            if i !=j:
                if corr.loc[i,j]>0.7: list_temp.append([i,j,corr.loc[i,j]])
#list_temp
corr_pd = pd.DataFrame(list_temp, columns = ['var1','var2','corr'])
print(corr_pd)
print(corr_pd.sort_values(by = ['corr'],ascending = False, inplace = False))

# after looking at corr keep all columns

SalePrice corr:
OverallQual    0.808658
GrLivArea      0.732094
SalePrice      1.000000
Name: SalePrice, dtype: float64

           var1          var2      corr
0     YearBuilt   GarageYrBlt  0.847337
1   TotalBsmtSF      1stFlrSF  0.829169
2      1stFlrSF   TotalBsmtSF  0.829169
3     GrLivArea  TotRmsAbvGrd  0.827989
4  TotRmsAbvGrd     GrLivArea  0.827989
5   GarageYrBlt     YearBuilt  0.847337
6    GarageCars    GarageArea  0.853655
7    GarageArea    GarageCars  0.853655
           var1          var2      corr
6    GarageCars    GarageArea  0.853655
7    GarageArea    GarageCars  0.853655
0     YearBuilt   GarageYrBlt  0.847337
5   GarageYrBlt     YearBuilt  0.847337
1   TotalBsmtSF      1stFlrSF  0.829169
2      1stFlrSF   TotalBsmtSF  0.829169
3     GrLivArea  TotRmsAbvGrd  0.827989
4  TotRmsAbvGrd     GrLivArea  0.827989


In [68]:
# return to three sets
train         = dataset.loc[id_train]
test          = dataset.loc[id_test ]

In [69]:
# form lists for encoding
columns_zero =[]
ohe_list  = [el for el in (columns_obj+columns_num_cat)                               if el not in (list_to_delete)]
sc_list   = [el for el in (columns_float+columns_num_range+columns_int_years+columns_int_numerical) if el not in list_to_delete]
norm_list = [el for el in (columns_zero)                                             if el not in list_to_delete]


In [70]:
check = [el for el in train_copy.columns if el not in (ohe_list+sc_list+norm_list+list_to_delete)]
check

['Id', 'SalePrice']

In [71]:
#encoders

ohe  = OneHotEncoder(drop = 'if_binary', sparse = False)
norm = Normalizer()
sc   = StandardScaler()
#norm = StandardScaler()

In [72]:
#train.columns,ohe_list

In [73]:
sw_cl_do =0
if sw_cl_do ==1:
# try new var "clusters"
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=7)
    transformers_cl  = ColumnTransformer(
                            [('ohe' ,ohe ,ohe_list ),
                             ('sc'  ,sc  ,sc_list  ),
                             ('norm',norm,norm_list)],
                            remainder = 'passthrough')

    train_cl = ohe.fit_transform(train[ohe_list])
    test_cl  = ohe.transform(test[ohe_list])
    train['cluster'] = kmeans.fit_predict(train_cl)
    test ['cluster'] = kmeans.predict(test_cl)
    ohe_list.append('cluster')



Try clustering

In [74]:
if sw_cl_do ==1: 
    train['cluster'].value_counts(), test['cluster'].value_counts() 

In [75]:
if sw_cl_do ==1: 
    stop

In [76]:
sw_cl =0
if sw_cl ==1:
    train.loc[train['cluster'] ==4,'cluster'] = 9
    train.loc[train['cluster'] ==1,'cluster'] = 9
    train.loc[train['cluster'] ==6,'cluster'] = 9
    train.loc[train['cluster'] ==2,'cluster'] = 9
    test.loc[test['cluster'] ==4,'cluster'] = 9
    test.loc[test['cluster'] ==2,'cluster'] = 9
    train['cluster'].value_counts(), test['cluster'].value_counts() 

Try PCA

In [77]:
from sklearn.decomposition import PCA
pca = PCA()

In [78]:
sw_pca =0
if sw_pca ==1:
#norm = StandardScaler()
    transformers_for_pca  = ColumnTransformer(
                            [('sc'  ,sc  ,sc_list  ),
                             ('norm',norm,norm_list)],
                            remainder = 'passthrough')

    train_for_pca = train[sc_list+norm_list]
    test_for_pca  = test [sc_list+norm_list]
    train_tr_pca = transformers_for_pca.fit_transform(train_for_pca )
    test_tr_pca  = transformers_for_pca.transform(test_for_pca )
    #print(test_for_pca.shape,test_tr_pca.shape)
    train_pca     = pca.fit_transform(train_tr_pca)
    test_pca      = pca.transform(test_tr_pca)
    train_pca_df  =pd.DataFrame(train_pca, 
                                columns = [f"PCA{i+1}" for i in range(train_pca.shape[1])],
                                index =train.index)
    test_pca_df   =pd.DataFrame(test_pca ,
                                columns = [f"PCA{i+1}" for i in range(train_pca.shape[1])],
                                index =test.index)
    train_cat     = train[ohe_list]
    test_cat      = test [ohe_list] 
    train         = pd.concat([train_cat,train_pca_df], axis =1)
    test          = pd.concat([test_cat ,test_pca_df ], axis =1)
    transformers  = ColumnTransformer(
                            [('ohe' ,ohe ,ohe_list ),],
                            remainder = 'passthrough')

In [79]:
if sw_pca ==0:
    # Prepare encodering
    ohe  = OneHotEncoder(drop = 'if_binary', sparse = False)
    norm = Normalizer()
    sc   = StandardScaler()

    transformers  = ColumnTransformer(
                            [('ohe' ,ohe ,ohe_list ),
                             ('sc'  ,sc  ,sc_list  ),
                             ('norm',norm,norm_list)],
                            remainder = 'passthrough')


In [80]:
gbr = GradientBoostingRegressor(max_depth = 4,n_estimators = 100,criterion = 'squared_error',random_state = 0)
svr = SVR()
knn = KNeighborsRegressor()
mlp = MLPRegressor()
# in previous culculations gbr shows best results amoung svr, knn, gbr
# so in this pipeline only gbr is tuned

In [81]:
pipe = Pipeline(steps = [('transformers',transformers),('clf', gbr)])

In [82]:
params = [{'clf':[gbr],'clf__max_depth':[3,4,5],
           'clf__learning_rate':[0.05,0.025],
           'clf__n_estimators' :[300,350,400],
           }
         #,
         #{'clf':[svr],'clf__C':[0.1,0.05,0.2]},
         #{'clf':[knn],'clf__n_neighbors':[3,4,5]},
         #{'clf':[mlp]}
         ]

In [83]:
y_train_log      = np.log(y)    

In [84]:
search = GridSearchCV(pipe,params,return_train_score = True, cv = 4)
search.fit(train,y_train_log)

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('transformers',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(drop='if_binary',
                                                                                       sparse=False),
                                                                         ['MSZoning',
                                                                          'Alley',
                                                                          'LotShape',
                                                                          'LandContour',
                                                                          'LotConfig',
                                                                          'LandSlope',
                                     

In [85]:
y_pred_test_log = search.predict(test)
y_pred_test_exp  = np.exp(y_pred_test_log)
y.mean(),y_pred_test_exp.mean()

(180979.11248285323, 177589.1771364407)

In [86]:
print(search.best_estimator_[1])
search.best_estimator_[1].get_params(deep =True)

GradientBoostingRegressor(criterion='squared_error', learning_rate=0.05,
                          n_estimators=400, random_state=0)


{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'init': None,
 'learning_rate': 0.05,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 400,
 'n_iter_no_change': None,
 'random_state': 0,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [87]:
search.best_score_

0.9019666681377867

In [88]:
best_model = search.best_estimator_[1]
best_model.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'init': None,
 'learning_rate': 0.05,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 400,
 'n_iter_no_change': None,
 'random_state': 0,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [89]:
type(train), train.shape,train.dtypes

(pandas.core.frame.DataFrame,
 (1458, 66),
 MSSubClass               int64
 MSZoning                object
 LotFrontage            float64
 LotArea                  int64
 Alley                   object
                         ...   
 MoSold                   int64
 YrSold                   int64
 SaleType                object
 SaleCondition           object
 LotFrontage_replace      int32
 Length: 66, dtype: object)

In [90]:
scorer = make_scorer(mean_squared_error)
score_cross = cross_val_score(best_model,
                              transformers.fit_transform(train,y_train_log),
                              y_train_log, cv =4, scoring = scorer )
score = np.sqrt(score_cross.mean())
print(score_cross,score)

[0.01176126 0.02168301 0.0148185  0.01467321] 0.12543521680306147


In [91]:
sw_log_to_csv =    1
if sw_log_to_csv ==1:
    
    y_pred_with_Id       = pd.DataFrame({'Id':id_test,'SalePrice':y_pred_test_exp})
    y_pred_with_Id.index = y_pred_with_Id['Id']
    y_pred_with_Id.to_csv("submission.csv", index = False)    